# nuScenes-lidarseg tutorial

Welcome to the nuScenes-lidarseg tutorial.

This demo assumes that nuScenes is installed at `/data/sets/nuscenes`. The mini version (i.e. v1.0-mini) of the full dataset will be used for this demo.

## Setup
To install the nuScenes-lidarseg expansion, please download the dataset from https://www.nuscenes.org/download. Upon unzipping, you should see three items:
- ```lidarseg``` 
- ```lidarseg_category.json```
- ```lidarseg.json```

Copy the `lidarseg` folder (this contains `.bin` files; each `.bin` file is an annotation for a pointcloud) into `/data/sets/nuscenes/`, and copy `lidarseg_category.json` and `lidarseg.json` into `/data/sets/nuscenes/v1.0-mini`.

Essentially, your `/data/sets/nuscenes` folder should end up looking like this:
```
└── nuscenes  
    ├── Usual nuscenes folders (i.e. samples, sweep)
    │
    ├── lidarseg <- Contains the .bin files 
    │               (each .bin file contains the label of each point in the point cloud)
    └── v1.0-mini
        ├── Usual files (e.g. attribute.json, calibrated_sensor.json etc.)     
        ├── lidarseg.json  <- contains the mapping of each .bin file to the token  
        └── lidarseg_category.json <- contains the categories of the labels
```

If you have downloaded `v1.0-trainval` and `v1.0-test` as well, then your `/data/sets/nuscenes` folder should look like this:
```
└── nuscenes  
    ├── Usual nuscenes folders (i.e. samples, sweep)
    │
    ├── lidarseg <- Contains the .bin files 
    │               (each .bin file contains the label of each point in the point cloud)
    │               Note that v1.0-test does not have any .bin files associated with it.
    ├── v1.0-mini
    │   ├── Usual files (e.g. attribute.json, calibrated_sensor.json etc.)     
    │   ├── lidarseg.json  <- contains the mapping of each .bin file to the token  
    │   └── lidarseg_category.json <- contains the categories of the labels
    │
    ├── v1.0-test
    │   ├── Usual files (e.g. attribute.json, calibrated_sensor.json etc.)     
    │   ├── lidarseg.json  <- contains the mapping of each .bin file to the token  
    │   └── lidarseg_category.json <- contains the categories of the labels
    │
    └── v1.0-trainval
        ├── Usual files (e.g. attribute.json, calibrated_sensor.json etc.)     
        ├── lidarseg.json  <- contains the mapping of each .bin file to the token  
        └── lidarseg_category.json <- contains the categories of the labels
```

## Initialization
Let's start by importing the necessary libraries:

In [ ]:
%matplotlib inline

from nuscenes import NuScenes

nusc = NuScenes(version='v1.0-test', dataroot='/data/sets/nuscenes', verbose=True)

As you can see, you do not need any extra libraries to use nuScenes-lidarseg. The original nuScenes devkit which you are familiar with has been extended so that you can use it seamless with nuScenes-lidarseg.

## Statistics of lidarseg dataset for the v1.0-mini split
Let's get a quick feel of the lidarseg dataset by looking at what classes are in it and the number of points belonging to each class. The classes will be sorted in ascending order based on the number of points.

In [ ]:
nusc.list_lidarseg_categories()

## Pick a sample token
Let's pick a sample token to use for this tutorial.

In [ ]:
my_sample = nusc.sample[87]

## Get statistics of a lidarseg sample token
Let's pick a lidarseg sample token and take a look at what objects are present in this particular sample.

In [ ]:
nusc.get_sample_lidarseg_stats(my_sample['token'], sort_counts=True)

By doing ```sort_counts=True```, the classes and their respective frequency counts are printed in ascending order; On the other hand, ```sort_counts=False``` will print the classes and their respective frequency counts in alphabetical order.

## Render the lidarseg labels in the bird's eye view of a pointcloud
In the original nuScenes devkit, you would pass a sample data token into ```render_sample_data``` to render a bird's eye view of the pointcloud. However, the points would be colored according to the distance from the ego vehicle. Now with the extended nuScenes devkit, all you need to do is set ```show_lidarseg_labels=True``` to visualize the class labels of the pointcloud.

In [ ]:
sample_data_token = my_sample['data']['LIDAR_TOP']
nusc.render_sample_data(sample_data_token,
                        with_anns=False,
                        show_lidarseg_labels=True)

But what if you wanted to focus on only certain classes? Given the statistics of the pointcloud printed out previously, let's say you are only interested in trucks and trailers. You could see the class indices belonging to those classes from the statistics and then pass an array of those indices into ```filter_lidarseg_labels``` like so:

In [ ]:
nusc.render_sample_data(sample_data_token,
                        with_anns=False,
                        show_lidarseg_labels=True,
                        filter_lidarseg_labels=[3, 4, 5])

Now only points in the pointcloud belonging to trucks and trailers are filtered out for your viewing pleasure. 

## Render lidarseg labels in image
If you wanted to superimpose the pointcloud into the corresponding image from a camera, you can use ```render_pointcloud_in_image``` like what you would do with the original nuScenes devkit, but set ```show_lidarseg_labels=True``` (remember to set ```render_intensity=False```). Similar to ```render_sample_data```, you can filter to see only certain classes using ```filter_lidarseg_labels```. And you can use ```show_lidarseg_legend``` to display a legend in the rendering.

In [ ]:
nusc.render_pointcloud_in_image(my_sample['token'],
                                pointsensor_channel='LIDAR_TOP',
                                camera_channel='CAM_BACK',
                                render_intensity=False,
                                show_lidarseg_labels=True,
                                filter_lidarseg_labels=[3, 4, 5],
                                show_lidarseg_legend=True,
                                verbose=True)

Say you happen to choose a camera channel where there are no points to show after filtering; you can set ```render_if_no_points=False``` so that nothing is displayed / saved in this case. For example, if we filter to see bicycles and objects which are pushable or pullable, and there are no points belonging to those classes, then:

In [ ]:
nusc.render_pointcloud_in_image(my_sample['token'],
                                pointsensor_channel='LIDAR_TOP',
                                camera_channel='CAM_BACK',
                                render_intensity=False,
                                show_lidarseg_labels=True,
                                filter_lidarseg_labels=[30, 15],
                                show_lidarseg_legend=True,
                                verbose=True,
                                render_if_no_points=False)

This might be useful when you want to render a bunch of sample tokens, but only for those which contain useful information (i.e. points belonging to certain classes) in them.

## Render sample (i.e. lidar, radar and all cameras)
Of course, like in the original nuScenes devkit, you can render all the sensors at once with ```render_sample```. In this extended nuScenes devkit, you can set ```show_lidarseg_labels=True``` to see the lidarseg labels. Similar to the above methods, you can use ```filter_lidarseg_labels``` to display only the classes you wish to see.

In [ ]:
nusc.render_sample(my_sample['token'],
                   show_lidarseg_labels=True,
                   filter_lidarseg_labels=[3, 4, 5],
                   verbose=True)

## Render a scene for a given camera sensor with lidarseg labels
You can also render an entire scene with the lidarseg labels for a camera of your choosing (the ```filter_lidarseg_labels``` and ```render_if_no_points``` arguments are present here as well).

Let's pick a scene first:

In [ ]:
my_scene = nusc.scene[0]

We then pass the scene token into ```render_camera_channel_with_pointclouds``` indicating that we are only interested in construction vehicles and man-made objects.

<img src="lidarseg_render_scene.gif" width="640" align="center">

(Note: the following code is commented out as it crashes in Jupyter notebooks.)

In [ ]:
# import os
nusc.render_camera_channel_with_pointclouds(my_scene['token'], 
                                            'CAM_BACK', 
                                            filter_lidarseg_labels=[6, 36],
                                            render_if_no_points=True, 
                                            verbose=True, 
                                            imsize=(1280, 720))

To save the renderings as images, you can pass a path to a folder you want to save the images to via the ```out_folder``` argument.

(Note: the following code is commented out as it crashes in Jupyter notebooks.)

In [ ]:
nusc.render_camera_channel_with_pointclouds(my_scene['token'], 'CAM_BACK',
                                            filter_lidarseg_labels=[6, 36],
                                            render_if_no_points=True,
                                            verbose=True,
                                            imsize=(1280, 720),
                                            out_folder=os.path.expanduser('~/Desktop/my_folder'))

Alternatively, to save the renderings into a video, all you have to do is pass a path ending in a ```.avi``` extension to ```out_folder```. Here, we filter to show only points belonging to construction vehicles.

(Note: the following code is commented out as it crashes in Jupyter notebooks.)

In [ ]:
nusc.render_camera_channel_with_pointclouds(my_scene['token'], 'CAM_BACK',
                                            filter_lidarseg_labels=[6],
                                            render_if_no_points=False,
                                            verbose=True,
                                            imsize=(1280, 720),
                                            out_folder=os.path.expanduser('~/Desktop/my_rendered_scene.avi'))

## Render a scene for all cameras with lidarseg labels
You can also render the entire scene for all cameras at once with the lidarseg labels as a video. Let's say in this case, we are interested in points belonging to driveable surfaces and cars.

<img src="lidarseg_render_scene_all_cams.gif" width="1280" align="center">

(Note: the following code is commented out as it crashes in Jupyter notebooks.)

In [ ]:
nusc.render_scene_with_pointclouds_for_all_cameras(my_scene['token'], 
                                                   filter_lidarseg_labels=[32, 1],
                                                   out_path=os.path.expanduser('~/Desktop/my_rendered_scene.avi'))

## Visualizing LIDAR segmentation predictions
In all the above functions, the labels of the LIDAR pointcloud which have been rendered are the ground truth. If you have trained a model to segment LIDAR pointclouds and have run it on the nuScenes-lidarseg dataset, you can visualize your model's predictions simply by using the `show_lidarseg_preds` argument in the above functions.

You simply need to pass the path to the folder where your predictions are stored into `show_lidarseg_preds`, like so:

In [ ]:
nusc.render_pointcloud_in_image(my_sample['token'],
                                pointsensor_channel='LIDAR_TOP',
                                camera_channel='CAM_BACK',
                                render_intensity=False,
                                show_lidarseg_labels=True,
                                filter_lidarseg_labels=[3, 4, 5],
                                show_lidarseg_legend=True,
                                verbose=True
                                show_lidarseg_preds='/data/exp/nuscenes_lidarseg/lidarseg_preds')

Note that each set of predictions in the folder must be a `.bin` file and named as `<lidar_sample_data_token>.bin`.

## Conclusion
And this brings us to the end of the tutorial for nuScenes-lidarseg, enjoy!